## EXERCICE 1

a) D'après la loi de Snell-Descartes :
$n_{\text{air}} \cdot \sin(\phi) = n_{c} \cdot \sin(\phi_{\text{r}})$

![image](./img_exo1.png)

Aussi on a: $n_{c} \sin(\theta_{c}) = n_{g} \sin\left(\frac{\pi}{2}\right)$
, avec $\theta_{c} = \frac{\pi}{2} - \phi_{r}$

Donc $n_{\text{air}} \cdot \sin(\phi) = n_{c} \cdot \cos(\theta_{\text{c}})$
,or $\sin(\theta_{c}) = \dfrac{n_{g}}{n_{c}}$

Ainsi: $\sin(\phi) = \sqrt{n_{c}^{2} - n_{g}^{2}}$

b) D'après le cours: $$\Delta T = \frac{n_{c}}{c} \cdot L \cdot \sqrt{\frac{1}{\sin(\theta_{c})} - 1}$$

Or $$\sin(\theta_{c}) = \sqrt{1 - \cos^2(\theta_{c})} = \sqrt{1 - \left(\frac{\sin(\phi)}{n_{c}}\right)^2}$$

D'où $$\Delta T = \frac{n_{c}}{c} \cdot l \cdot \frac{1}{\sqrt{1 - \left(\frac{\sin(\phi)}{n_{c}}\right)^2}} - 1$$

c) C'est la dispersion modale

d) $$T_{b} = \frac{1}{D} > \Delta T$$
,donc $$D_{max} = \frac{1}{\Delta T}$$

e) $$\Delta T = 3.29 µs$$








## EXERCICE 2

$$V = \frac{2 \pi a}{\lambda} \times \sqrt{n_{c}^{2} - n_{g}^{2}}=\frac{d \pi }{\lambda} \times \sqrt{n_{c}^{2} - n_{g}^{2}}$$

La condition pour que la fibre soit monomode est :$$V <2.405 $$ 
Donc, $$d=\frac{\lambda}{\pi \cdot \sqrt{n_{c}^{2} - n_{g}^{2}}}=6.5 µm$$


## EXERCICE 3
 
a) $P_{s}=P_{e}-A=-10-30=-40\text{ dBm}=10^{-7}\text{ W}$

b)$$L=\frac{A}{A_{linéique}}=150 \text{ km}$$



## EXERCICE 4

a) La structure 6 permet de proteger la fibre optique de l'eau

b)L'eau interfère avec la transmission de lumière dans les fibres optiques en raison de sa différence d'indice de réfraction par rapport au verre des fibres. Cette interférence peut entraîner des pertes de signal et altérer la qualité de transmission des données. 

c)On aura besoin : d'amplificateurs optiques

## EXERCICE 5
On cherche à résoudre l'équation de l'enveloppe de l'onde propagé suivante :

$$\dfrac{\partial A}{\partial z} (z,t) = -i\dfrac{\beta_2}{2} \dfrac{\partial^2 A}{\partial t^2} (z,t) $$
tel que :
$$\beta_2 = -\dfrac{D\lambda^2}{2\pi c} $$

On applique la transformée de Fourier à cette équation :
$$\mathcal{F}\left\{ \dfrac{\partial A}{\partial z} \right\}(z,\omega) = -i\dfrac{\beta_2}{2} \mathcal{F}\left\{\dfrac{\partial^2 A}{\partial t^2} \right\}(z,\omega) \Longrightarrow 
\dfrac{\partial }{\partial z} \mathcal{F}\left\{ A \right\}(z,\omega) = -i\dfrac{\beta_2}{2} (-i\omega)^2 \mathcal{F}\left\{ A \right\}(z,\omega)
$$

Ce qui donne :
$$ \dfrac{\partial \hat{A} }{\partial z} (z,\omega) = \dfrac{i\beta_2 \omega^2}{2}\hat{A}(z,\omega) $$

D'où la solution :
$$ \hat{A}(z,\omega) = \hat{A}(0,\omega)e^{i\frac{\beta_2\omega^2}{2}z} $$

In [1]:
import numpy as np
from astropy import units as u
from astropy import constants as const
import matplotlib.pyplot as plt

In [2]:
# Physical parameters
λ = 1.55 * u.um                             # wavelength
D = 17.0 * u.ps / (u.nm * u.km)             # Dispersion Coefficient
β_2 = - D * λ**2 / (2 * np.pi * const.c)    # Group velocity dispersion parameter
B = 50 * u.Gbit / u.s                       # Bit rate : 1kbps, 100 Mbps, 1 Gbps, 10 Gbps, 50 Gbps
B_nounit = B.to(u.bit/u.s)/(u.bit/u.s)      # Capacity Hertz unit
L = 1 * u.km                                # Fiber Lenght
T = 0.5 * u.s                               # Total duration of the simulation

In [ ]:
I_max = 40 * u.mA # maximum intensity value of diode laser 
I_nounit = I_max.to(u.A)/(1 * u.A)


bits_seq = np.random.randint(2, size=80) # np.array([0,1,0]) #   
bits_sended_lenght = len(bits_seq)

τ = .375 / B # caracteristic time of the gaussian pulse
τ_nounit = τ.to(u.s/u.bit)/(1 * u.s/u.bit)

def generate_gaussian_pulse(duration, sampling_rate):
    t = np.arange(0, duration, 1/sampling_rate) 
    pulse = np.sqrt(I_nounit) * np.array([np.sum([bits_seq[k] * np.exp(-((ti-k/B_nounit)/τ_nounit)**2) for k in range(bits_sended_lenght)]) for ti in t])
    return t, pulse



Π = lambda t: 1.0 if np.abs(t)<.5 else 0.0 # Unitary rectangular window
def generate_rectangular_pulses(duration, sampling_rate):
    t = np.arange(0, duration, 1/sampling_rate) 
    pulse = np.sqrt(I_nounit) * np.array([np.sum([bits_seq[k] * Π((ti * B_nounit - k)) for k in range(bits_sended_lenght)]) for ti in t])
    return time, pulse


###  TEMPOREL ###
# Paramètres du pulse
duration = 2 * bits_sended_lenght * τ_nounit   # time window (in seconds)
sampling_rate = 4 * B_nounit  # sampling frequency (in Hz)

# Générer le pulse gaussien
time, gaussian_pulse = generate_gaussian_pulse(duration, sampling_rate)

# Générer le pulse rectangulaire
time, rect_pulse = generate_rectangular_pulses(duration, sampling_rate)






# Tracer le résultat
plt.figure(figsize=(10, 7))
plt.subplot(2,2,1)
plt.plot(time * B_nounit, gaussian_pulse, label='Gaussian Pulse')
plt.title(f'Before propagation') #bits_seq
plt.xlabel(r'$t/T_s$')
plt.legend()
plt.grid(True)


plt.subplot(2,2,3)
plt.plot(time * B_nounit, rect_pulse, label='Rect pulses')
plt.title(f'Before propagation') #bits_seq
plt.xlabel(r'$t/T_s$')
plt.legend()
plt.grid(True)



###  SPECTRAL ###

"""
freq = np.fft.fftfreq(len(time), d=1/sampling_rate)
ω = 2 * np.pi * freq * u.Hz
fft_gauss_pulse = np.fft.fft(gaussian_pulse) 
fft_rect_pulse = np.fft.fft(rect_pulse)
"""



plt.subplot(2,2,2)
# plt.plot(freq, np.abs(fft_gauss_pulse), label=r'$|\hat{A}(\nu)|$')
plt.psd(gaussian_pulse, Fs=sampling_rate.value, sides='twosided')
plt.xlabel(r'$\nu$ (Hz)')
plt.grid(True)


plt.subplot(2,2,4)
#plt.plot(freq, np.abs(fft_rect_pulse), label=r'$|\hat{A}(\nu)|$')
plt.psd(rect_pulse, Fs=sampling_rate.value, sides='twosided', scale_by_freq=False)
plt.xlabel(r'$\nu$ (Hz)')
plt.grid(True)


plt.tight_layout()
plt.show()

In [4]:
def eye_diagram(data, sampling_rate,  time_eye_window_max):
    data_duration = len(data)/sampling_rate
    data_time = np.arange(0,data_duration, 1/sampling_rate)
    window = np.arange(0, time_eye_window_max, 1/sampling_rate)
    window[-1]=np.nan
    time_eye_value = np.array([window for i in range(int(np.floor(data_duration/time_eye_window_max)))]).flatten()
    time_eye_value = np.concatenate((time_eye_value, window[0 : len(data_time) - len(time_eye_value)]))

    plt.plot(time_eye_value * B_nounit, data)
    #plt.plot(time * B_nounit, rect_pulse, label='Rect pulses')
    #plt.title(f'Before propagation : Eye diagram') 
    plt.xlabel(r'$t$ [s]')
    #plt.ylabel('Envelop')
    plt.legend()
    plt.grid(True)

In [5]:
freq = np.fft.fftfreq(len(time), d=1/sampling_rate)

#ω = 2 * np.pi * const.c/λ
ω = 2 * np.pi * freq * u.Hz
fft_gauss_pulse_prop = np.fft.fft(gaussian_pulse) * np.exp(1j * (β_2 * ω**2 * L/2).to(u.dimensionless_unscaled))
fft_rect_pulse_prop = np.fft.fft(rect_pulse) * np.exp(1j * (β_2 * ω**2 * L/2).to(u.dimensionless_unscaled))


gauss_pulse_prop = np.abs(np.fft.ifft(fft_gauss_pulse_prop))
rect_pulse_prop = np.abs(np.fft.ifft(fft_rect_pulse_prop))

In [ ]:
eye_diagram(gauss_pulse_prop, sampling_rate, 4 * 1/B_nounit)